In [0]:
# Classes for different data sources
class DataSink():                 # Abstract class
    def __init__(self, df, path, method, params):
        self.df = df
        self.path = path
        self.method = method        # input to spark.write.mode like overwrite, etc
        self.params = params

    def load_dataframe(self):        # Abstract method to be implemented in the subclasses
        raise ValueError(f"Abstract Method \"load_dataframe\" has not been implemented")

class loadToDBFS(DataSink):
    def load_dataframe(self):
        self.df.write.mode(self.method).save(self.path)     
        # Since .format is not used, data is stored in the delta table form by default (online says should be parquet)
        
class loadToDBFSWithPartition(DataSink):
    def load_dataframe(self):
        partitionByColumns = self.params.get("partitionByColumns")      # List of columns to partition by
        self.df.write.mode(self.method).partitionBy(*partitionByColumns).save(self.path)
        # Since .format is not used, data is stored in the delta table form by default (online says should be parquet)

class loadToDeltaTable(DataSink):
    def load_dataframe(self):
        self.df.write.format("delta").mode(self.method).saveAsTable(self.path)

In [0]:
def get_data_sink(sink_type, df, path, method, params = None):
    if(sink_type == "dbfs"):
        return loadToDBFS(df, path, method, params)
    elif(sink_type == "dbfs_with_partition"):
        return loadToDBFSWithPartition(df, path, method, params)
    elif(sink_type == "delta"):
        return loadToDeltaTable(df, path, method, params)
    else:
        return ValueError(f"Data sink has not been implemented for sink type : {sink_type}")